# Link Para o Vídeo: https://youtu.be/ZTMBeikvW0A

In [9]:
import pandas as pd
import duckdb

# a) Ler o dataset fakeTelegram.BR_2022.csv.

In [3]:
df = pd.read_csv('fakeTelegram.BR_2022.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557586 entries, 0 to 557585
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   date_message            557586 non-null  object 
 1   id_member_anonymous     234245 non-null  object 
 2   id_group_anonymous      557586 non-null  object 
 3   media                   332605 non-null  object 
 4   media_type              332605 non-null  object 
 5   media_url               157445 non-null  object 
 6   has_media               557586 non-null  bool   
 7   has_media_url           557586 non-null  bool   
 8   trava_zap               557586 non-null  bool   
 9   text_content_anonymous  444201 non-null  object 
 10  dataset_info_id         557586 non-null  int64  
 11  date_system             557586 non-null  object 
 12  score_sentiment         444157 non-null  float64
 13  score_misinformation    167238 non-null  float64
 14  id_message          

# b) Remova os trava-zaps.

In [4]:
df['trava_zap'].value_counts()

trava_zap
False    557570
True         16
Name: count, dtype: int64

In [ ]:
df = df.loc[df['trava_zap'] == False]

# c) Exportar os dados para um arquivo Parquet.

In [8]:
df.to_parquet("fake_telegram.parquet", engine="pyarrow", index=False)

# d) Exportar os dados para o DuckDB.

In [ ]:
con = duckdb.connect("data_science-ufc.duckdb")

con.execute("CREATE TABLE fakeTelegram AS SELECT * FROM df")

# e) Utlizando o DuckDB recupere:

### e) 1. A quantidade de mensagens;

In [16]:
con.execute("""
            SELECT COUNT(text_content_anonymous) FROM fakeTelegram
            """).fetchdf()

,count(text_content_anonymous)
0,444201


### 2. A quantidade de usuários;

In [18]:
con.execute("""
            SELECT COUNT(id_member_anonymous) FROM fakeTelegram
            """).fetchdf()

,count(id_member_anonymous)
0,234233


### 3. A quantidade de grupos;

In [20]:
con.execute("""
            SELECT COUNT(id_group_anonymous) FROM fakeTelegram
            """).fetchdf()

,count(id_group_anonymous)
0,557570


### 4. Quantidade de mensagens que possuem apenas texto;

In [22]:
con.execute("""
            SELECT COUNT(*) 
			FROM fakeTelegram
			WHERE text_content_anonymous IS NOT NULL 
			AND has_media_url = FALSE;
            """).fetchdf()

,count_star()
0,286756


### 5. Quantidade de mensagens contendo mídias;

In [26]:
con.execute("""
            SELECT COUNT(*) 
			FROM fakeTelegram
			WHERE text_content_anonymous IS NOT NULL 
			AND has_media_url = TRUE;
            """).fetchdf()

,count_star()
0,157445


### 6. Quantidade de mensagens por tipo de mídia (jpg, mp4 etc);

In [37]:
con.execute("""
            SELECT media_type, COUNT(*) AS total_media
			FROM fakeTelegram
			WHERE media_type IS NOT NULL
			GROUP BY media_type
			ORDER BY total_media;
            """).fetchdf()

,media_type,total_media
0,image/wma,1
1,image/m4a,1
2,application/binary,1
3,image/7f9d809d82c2545a2dc8160d5015c8ad,1
4,application/vnd.ms-xpsdocument,1
...,...,...
62,application/pdf,2850
63,application/vnd.android.package-archive,7159
64,video/mp4,18497
65,url,100856


In [ ]:
con.execute("""
            SELECT split_part(media_type, '/', 1) AS tipo, COUNT(*) AS total_media
			FROM fakeTelegram
			WHERE media_type IS NOT NULL
			GROUP BY tipo
			ORDER BY total_media;
            """).fetchdf()

,tipo,total_media
0,text,3
1,audio,177
2,application,10168
3,video,18631
4,url,100856
5,image,202769


In [41]:
con.execute("""
            SELECT split_part(media_type, '/', 2) AS tipo, COUNT(*) AS total_media
			FROM fakeTelegram
			WHERE media_type IS NOT NULL
			AND tipo IS NOT NULL
			GROUP BY tipo
			ORDER BY total_media;
            """).fetchdf()

,tipo,total_media
0,gif,1
1,7f9d809d82c2545a2dc8160d5015c8ad,1
2,x-ms-wma,1
3,gz,1
4,binary,1
...,...,...
56,pdf,3080
57,vnd.android.package-archive,7159
58,mp4,20088
59,,100856


### 7, 8 e 9. (Cancelado)

### 10. Quantidade de mensagens por país;
### 11. Quantidade de mensagens Brasil X Países Estrangeiros;

### 12. As 30 URLs que mais se repetem (mais compartilhadas);

In [ ]:
con.execute("""
            SELECT media_url, COUNT(media_url) AS qtde_url
			FROM fakeTelegram
			WHERE media_url IS NOT NULL
			GROUP BY media_url
			ORDER BY qtde_url DESC LIMIT 30;
            """).fetchdf()

,media_url,qtde_url
0,t.me/alexeconomia,4159
1,https://youtube.com/c/especulandoosfatosoficial,1607
2,t.me/fimtaproximo,1581
3,t.me/+EWlGMatRZGg3OTlh,1419
4,https://youtu.be/qbTzhB0akt8,1160
5,https://youtu.be/zDuOoyhyN-4,1022
6,t.me/+ewZIPdZ42vEyNzJh,772
7,https://t.me/canalselvabrasiloficial,709
8,https://youtu.be/4DHk9KZ01HM,660
9,T.me/Arthur_Senna_Trader2,640


### 13. Os 30 domínios que mais se repetem (mais compartilhados);

In [ ]:
con.execute("""
    SELECT lower(regexp_extract(media_url, '^(?:https?://)?(?:www\\.)?([^/]+)', 1)) AS dominio, COUNT(*) AS total
    FROM fakeTelegram
    WHERE media_url IS NOT NULL
    GROUP BY dominio
    ORDER BY total DESC
    LIMIT 30;
""").fetchdf()


,dominio,total
0,youtu.be,52842
1,t.me,21367
2,youtube.com,14768
3,terrabrasilnoticias.com,8284
4,instagram.com,6668
5,jornaldacidadeonline.com.br,4829
6,twitter.com,4442
7,gazetabrasil.com.br,1781
8,pleno.news,1602
9,facebook.com,1368


In [56]:
corrigir_dominios = {
    "youtube.com": "YouTube",
    "m.youtube.com": "YouTube",
    "youtu.be": "YouTube",
    "t.me": "Telegram",
    "T.me": "Telegram",
    "facebook.com": "Facebook",
    "fb.watch": "Facebook",
    "instagram.com": "Instagram",
    "twitter.com": "Twitter",
    "chat.whatsapp.com": "WhatsApp",
}



df_dominios = con.execute("""
    SELECT lower(regexp_extract(media_url, '^(?:https?://)?(?:www\\.)?([^/]+)', 1)) AS dominio, COUNT(*) AS total
    FROM fakeTelegram
    WHERE media_url IS NOT NULL
    GROUP BY dominio
    ORDER BY total DESC
    LIMIT 30;
	""").fetchdf()


df_dominios["plataforma"] = df_dominios["dominio"].map(corrigir_dominios).fillna(df_dominios["dominio"])

df_agregado = df_dominios.groupby("plataforma", as_index=False)["total"].sum().sort_values("total", ascending=False)

df_agregado.head(30)


,plataforma,total
5,YouTube,68304
2,Telegram,21367
21,terrabrasilnoticias.com,8284
1,Instagram,6668
14,jornaldacidadeonline.com.br,4829
3,Twitter,4442
0,Facebook,1941
12,gazetabrasil.com.br,1781
17,pleno.news,1602
8,bitchute.com,1178


### 14. Os 30 usuários mais ativos;

In [61]:
con.execute(""" 
	SELECT id_member_anonymous AS users, COUNT(*) AS count_atividade
	FROM fakeTelegram
	WHERE id_member_anonymous IS NOT NULL
	AND (text_content_anonymous IS NOT NULL OR media IS NOT NULL)
	GROUP BY users
	ORDER BY count_atividade DESC
	LIMIT 30;
	""").fetchdf()

,users,count_atividade
0,abe534d581ec6d552243d6955d3c3cd8,12289
1,1665e22b0f564cd46d343f7677014821,5452
2,1ac091b8ed5c4e42383f1b4ff4cc9b2d,5060
3,c743967449a387ad2c1c7e03b2c45b36,3019
4,e7998863ac2a40086657fab4a6b463c9,1928
5,e8fd8fee8c39342d37993775da7756d5,1706
6,2f4be6244ede15b46e8329a2c975be30,1620
7,d49f81df0c75d1d72bee6c5b2d707da0,1571
8,4f7d493f0f6222d56e5b19a4f7c336cc,1447
9,773b9bd5b02a2e96f9d732c29bfcb663,1411


### 15. Os 30 usuários que mais compartilharam texto;

In [62]:
con.execute(""" 
    SELECT id_member_anonymous as users, COUNT(text_content_anonymous) as count_messages
    FROM fakeTelegram
    WHERE id_member_anonymous IS NOT NULL
    GROUP BY id_member_anonymous
    ORDER BY count_messages DESC
    LIMIT 30;
	""").fetchdf()

,users,count_messages
0,abe534d581ec6d552243d6955d3c3cd8,12289
1,1665e22b0f564cd46d343f7677014821,5452
2,1ac091b8ed5c4e42383f1b4ff4cc9b2d,4022
3,c743967449a387ad2c1c7e03b2c45b36,3019
4,d49f81df0c75d1d72bee6c5b2d707da0,1571
5,e7998863ac2a40086657fab4a6b463c9,1401
6,4f7d493f0f6222d56e5b19a4f7c336cc,1365
7,773b9bd5b02a2e96f9d732c29bfcb663,1313
8,e8fd8fee8c39342d37993775da7756d5,1225
9,f233cf8b1d4ede06f32199a7e0081bf5,1149


### 16. Os 30 usuários que mais compartilharam mídias;

In [64]:
con.execute(""" 
			SELECT id_member_anonymous AS users, COUNT(*) AS count_midia
			FROM fakeTelegram
			WHERE has_media = TRUE
			AND id_member_anonymous IS NOT NULL
			GROUP BY users
			ORDER BY count_midia DESC
			LIMIT 30;
			""").fetchdf()

,users,count_midia
0,1ac091b8ed5c4e42383f1b4ff4cc9b2d,4632
1,1665e22b0f564cd46d343f7677014821,2972
2,e7998863ac2a40086657fab4a6b463c9,1498
3,2f4be6244ede15b46e8329a2c975be30,1489
4,4f7d493f0f6222d56e5b19a4f7c336cc,1359
5,c052c859b42c5a1923c22f5a201de746,1289
6,773b9bd5b02a2e96f9d732c29bfcb663,1167
7,56b8359fd127312651b80b8ed8030085,1079
8,e8fd8fee8c39342d37993775da7756d5,1007
9,3e49fd40fd973ee1b8f1a6d58feb4a54,965


### 17. As 30 mensagens mais compartilhadas;

In [76]:
con.execute(r"""
			SELECT normalized_text, COUNT(*) AS total
			FROM (
				SELECT 
					trim(
						regexp_replace(
							regexp_replace(
								regexp_replace(
									regexp_replace(
										regexp_replace(
											regexp_replace(
												regexp_replace(
													regexp_replace(
														regexp_replace(
															lower(text_content_anonymous),
															'[áàãâä]', 'a', 'g'
														),
														'[éèêë]', 'e', 'g'
													),
													'[íìîï]', 'i', 'g'
												),
												'[óòõôö]', 'o', 'g'
											),
											'[úùûü]', 'u', 'g'
										),
										'[ç]', 'c', 'g'
									),
									'[^a-z0-9 ]', '', 'g'
								),
								'\\s+', ' ', 'g'
							),
							'^\\s+|\\s+$', '', 'g'  -- remove espaços extras no início/fim
						)
					) AS normalized_text
				FROM fakeTelegram
				WHERE text_content_anonymous IS NOT NULL
			)
			GROUP BY normalized_text
			ORDER BY total DESC
			LIMIT 30;
			""").fetchdf()


,normalized_text,total
0,this community was blocked in brazil following...,17422
1,,2444
2,roughsex,1134
3,anal sex,1118
4,rough,1041
5,vpn vpn taqiqlangan saytlardan ...,1019
6,httpsyoutubeqbtzhb0akt8,765
7,httpsyoutubezduooyhyn4,715
8,amerikalik ayolni arab erkaklari tomonidan zor...,632
9,foto de nelia barros,548


### 18. As 30 mensagens mais compartilhadas em grupos diferentes;

In [84]:
con.execute(r"""
            SELECT normalized_text, COUNT(DISTINCT id_group_anonymous) AS total_grupos
            FROM (
                SELECT 
                    regexp_replace(
                        regexp_replace(
                            regexp_replace(
                                regexp_replace(
                                    regexp_replace(
                                        regexp_replace(
                                            regexp_replace(
                                                regexp_replace(
                                                    regexp_replace(
                                                        lower(text_content_anonymous),
                                                        '[áàãâä]', 'a', 'g'
                                                    ),
                                                    '[éèêë]', 'e', 'g'
                                                ),
                                                '[íìîï]', 'i', 'g'
                                            ),
                                            '[óòõôö]', 'o', 'g'
                                        ),
                                        '[úùûü]', 'u', 'g'
                                    ),
                                    '[ç]', 'c', 'g'
                                ),
                                '[^a-z0-9 ]', '', 'g'
                            ),
                            '\\s+', ' ', 'g'
                        ),
                        '^\\s+|\\s+$', '', 'g'
                    ) AS normalized_text,
                    id_group_anonymous
                FROM fakeTelegram
                WHERE text_content_anonymous IS NOT NULL
                AND id_group_anonymous IS NOT NULL
            )
            WHERE length(normalized_text) > 0
            GROUP BY normalized_text
            ORDER BY total_grupos DESC
            LIMIT 30;
            """).fetchdf()


,normalized_text,total_grupos
0,this community was blocked in brazil following...,59
1,user,39
2,boa noite a todos o que eu vou dizer e serio a...,36
3,hoje as 18 horas no canal inteligencia ltda do...,35
4,httpsyoutubezduooyhyn4,31
5,atencao chamada urgente patriotas do brasil d...,30
6,httpsyoutubeqbtzhb0akt8,30
7,os ptistas nao quer que agente mostre que voto...,29
8,de nada ira adiantar fiscalizar as urnas se na...,29
9,ajudem a subir a janonesamigodepedofilopor favor,29


### 19. Mensagens idênticas compartilhadas pelo mesmo usuário (e suas quantidades);

In [86]:
con.execute(r"""
			SELECT id_member_anonymous, normalized_text, COUNT(*) AS total_repeticoes
			FROM (
				SELECT 
					id_member_anonymous,
					regexp_replace(
						regexp_replace(
							regexp_replace(
								regexp_replace(
									regexp_replace(
										regexp_replace(
											regexp_replace(
												regexp_replace(
													regexp_replace(
														lower(text_content_anonymous),
														'[áàãâä]', 'a', 'g'
													),
													'[éèêë]', 'e', 'g'
												),
												'[íìîï]', 'i', 'g'
											),
											'[óòõôö]', 'o', 'g'
										),
										'[úùûü]', 'u', 'g'
									),
									'[ç]', 'c', 'g'
								),
								'[^a-z0-9 ]', '', 'g'
							),
							'\\s+', ' ', 'g'
						),
						'^\\s+|\\s+$', '', 'g'
					) AS normalized_text
				FROM fakeTelegram
				WHERE text_content_anonymous IS NOT NULL
				AND id_member_anonymous IS NOT NULL
			)
			WHERE length(normalized_text) > 0
			GROUP BY id_member_anonymous, normalized_text
			HAVING COUNT(*) > 1
			ORDER BY total_repeticoes DESC;
			""").fetchdf()


,id_member_anonymous,normalized_text,total_repeticoes
0,f233cf8b1d4ede06f32199a7e0081bf5,this community was blocked in brazil following...,838
1,c743967449a387ad2c1c7e03b2c45b36,this community was blocked in brazil following...,532
2,7696d5103cdb8ac352d748a1db1126b0,this community was blocked in brazil following...,530
3,1665e22b0f564cd46d343f7677014821,bem vindoa ao grupo ipira noticias compartilhe...,357
4,a398999c55f8d6f0c65760522ae12e45,this community was blocked in brazil following...,308
...,...,...,...
18934,4a498818da925377eff2606a260cfa45,espaol estoy escuchando el titulo de radio eva...,2
18935,27f5833380581fab3453c1acd26427d5,carla zambelli agora sobre orelatorio prazo de...,2
18936,ceab16e73b2bc8e07b0e4e3f20920992,assista a bo a volta dos mortos vivos no youtu...,2
18937,3bc8b1ec90f8f23e38a6d953e8faab61,o ministro disse que a analise das forcas arma...,2


### 20. Mensagens idênticas compartilhadas pelo mesmo usuário em grupos distintos (e suas quantidades);

In [90]:
con.execute(r"""
			SELECT id_member_anonymous, normalized_text, COUNT(DISTINCT id_group_anonymous) AS total_grupos
			FROM (
				SELECT 
					id_member_anonymous,
					id_group_anonymous,
					regexp_replace(
						regexp_replace(
							regexp_replace(
								regexp_replace(
									regexp_replace(
										regexp_replace(
											regexp_replace(
												regexp_replace(
													regexp_replace(
														lower(text_content_anonymous),
														'[áàãâä]', 'a', 'g'
													),
													'[éèêë]', 'e', 'g'
												),
												'[íìîï]', 'i', 'g'
											),
											'[óòõôö]', 'o', 'g'
										),
										'[úùûü]', 'u', 'g'
									),
									'[ç]', 'c', 'g'
								),
								'[^a-z0-9 ]', '', 'g'
							),
							'\\s+', ' ', 'g'
						),
						'^\\s+|\\s+$', '', 'g'
					) AS normalized_text
				FROM fakeTelegram
				WHERE text_content_anonymous IS NOT NULL
				AND id_member_anonymous IS NOT NULL
				AND id_group_anonymous IS NOT NULL
			)
			WHERE length(normalized_text) > 0
			GROUP BY id_member_anonymous, normalized_text
			HAVING COUNT(DISTINCT id_group_anonymous) > 1
			ORDER BY total_grupos DESC;
			""").fetchdf()


,id_member_anonymous,normalized_text,total_grupos
0,a7840e7844020149e197272748965862,alertaesse grupo foi criado pela esquerda com ...,16
1,326d0a2f1bc5a1ede446fcf5dc31ff2e,urgentissimo esse grupo e uma fraude saiam d...,14
2,326d0a2f1bc5a1ede446fcf5dc31ff2e,bomba bombavcs estao achando que irao consegui...,12
3,29dda76df3384c28a07e8bad1ee9ceed,boa noite a todos o que eu vou dizer e serio a...,11
4,4e93f1d3c83fd6041314517363b14ed5,pessoal nao envie seus comprovantes de votacao...,10
...,...,...,...
13195,8b890330b9d851d740d3536ac1567432,a midia e tao imunda a mensagem nao saiu de mo...,2
13196,048fad9af3d2c9c92e04cfb0f0231e29,httpsyoutubecomshortsgwswyztkk2kfeatureshare,2
13197,2e08ae4c6ec1973e7ec515d3b0ba50d0,caso aconteceu na tarde desta segundafeira sai...,2
13198,4f7d493f0f6222d56e5b19a4f7c336cc,httpswwwyoutubecomwatchvcdajyvykxfk,2


### 21. Os 30 unigramas, bigramas e trigramas mais compartilhados;

Unigrama

In [110]:
con.execute(r"""
			WITH mensagens_norm AS (
			SELECT 
				regexp_replace(
				regexp_replace(
					regexp_replace(
					regexp_replace(
						regexp_replace(
						regexp_replace(
							regexp_replace(
							regexp_replace(
								regexp_replace(
								lower(text_content_anonymous),
								'[áàãâä]', 'a', 'g'
								),
								'[éèêë]', 'e', 'g'
							),
							'[íìîï]', 'i', 'g'
							),
							'[óòõôö]', 'o', 'g'
						),
						'[úùûü]', 'u', 'g'
						),
						'[ç]', 'c', 'g'
					),
					'[^a-z0-9 ]', '', 'g'
					),
					'\\s+', ' ', 'g'
				),
				'^\\s+|\\s+$', '', 'g'
				) AS normalized_text
			FROM fakeTelegram
			WHERE text_content_anonymous IS NOT NULL
			),
			tokenized AS (
			SELECT 
				unnest(string_split(normalized_text, ' ')) AS token
			FROM mensagens_norm
			),
			limpos AS (
			SELECT lower(token) AS unigram
			FROM tokenized
			WHERE token IS NOT NULL
				AND NOT token ~ '^[0-9]+$'
			)
			SELECT unigram, COUNT(*) AS total
			FROM limpos
			GROUP BY unigram
			ORDER BY total DESC
			LIMIT 31;
			""").fetchdf()


,unigram,total
0,,842025
1,e,489250
2,de,448411
3,a,352676
4,o,351619
5,que,297897
6,do,197214
7,nao,160421
8,para,155563
9,no,124808


Bigrama

In [ ]:
con.execute(r"""
			WITH mensagens_norm AS (
			SELECT 
				regexp_replace(
				regexp_replace(
					regexp_replace(
					regexp_replace(
						regexp_replace(
						regexp_replace(
							regexp_replace(
							regexp_replace(
								regexp_replace(
								lower(text_content_anonymous),
								'[áàãâä]', 'a', 'g'
								),
								'[éèêë]', 'e', 'g'
							),
							'[íìîï]', 'i', 'g'
							),
							'[óòõôö]', 'o', 'g'
						),
						'[úùûü]', 'u', 'g'
						),
						'[ç]', 'c', 'g'
					),
					'[^a-z0-9 ]', '', 'g'
					),
					'\\s+', ' ', 'g'
				),
				'^\\s+|\\s+$', '', 'g'
				) AS normalized_text
			FROM fakeTelegram
			WHERE text_content_anonymous IS NOT NULL
			),
			tokenized AS (
			SELECT
				row_number() OVER () AS msg_id,
				unnest(string_split(normalized_text, ' ')) AS token,
				generate_subscripts(string_split(normalized_text, ' '), 1) AS pos
			FROM mensagens_norm
			),
			ngrams AS (
			SELECT
				t1.token AS token,
				t2.token AS next_token
			FROM tokenized t1
			JOIN tokenized t2 
				ON t1.msg_id = t2.msg_id AND t1.pos + 1 = t2.pos
			WHERE 
				t1.token IS NOT NULL AND 
				t2.token IS NOT NULL AND
				NOT t1.token ~ '^[0-9]+$' AND 
				NOT t2.token ~ '^[0-9]+$'
			),
			bigrams AS (
			SELECT 
				lower(token || ' ' || next_token) AS bigrama
			FROM ngrams
			)
			SELECT bigrama, COUNT(*) AS total
			FROM bigrams
			GROUP BY bigrama
			ORDER BY total DESC
			LIMIT 31;
			""").fetchdf()


,bigrama,total
0,,429490
1,o que,27391
2,e o,25274
3,que o,23894
4,e a,21232
5,of the,18375
6,in brazil,17477
7,a decision,17436
8,electoral court,17428
9,superior electoral,17426


Trigrama

In [107]:
con.execute(r"""
			WITH mensagens_norm AS (
			SELECT 
				regexp_replace(
				regexp_replace(
					regexp_replace(
					regexp_replace(
						regexp_replace(
						regexp_replace(
							regexp_replace(
							regexp_replace(
								regexp_replace(
								lower(text_content_anonymous),
								'[áàãâä]', 'a', 'g'
								),
								'[éèêë]', 'e', 'g'
							),
							'[íìîï]', 'i', 'g'
							),
							'[óòõôö]', 'o', 'g'
						),
						'[úùûü]', 'u', 'g'
						),
						'[ç]', 'c', 'g'
					),
					'[^a-z0-9 ]', '', 'g'
					),
					'\\s+', ' ', 'g'
				),
				'^\\s+|\\s+$', '', 'g'
				) AS normalized_text
			FROM fakeTelegram
			WHERE text_content_anonymous IS NOT NULL
			),
			tokenized AS (
			SELECT
				row_number() OVER () AS msg_id,
				unnest(string_split(normalized_text, ' ')) AS token,
				generate_subscripts(string_split(normalized_text, ' '), 1) AS pos
			FROM mensagens_norm
			),
			ngrams AS (
			SELECT
				t1.token AS token,
				t2.token AS next1,
				t3.token AS next2
			FROM tokenized t1
			JOIN tokenized t2 ON t1.msg_id = t2.msg_id AND t1.pos + 1 = t2.pos
			JOIN tokenized t3 ON t1.msg_id = t3.msg_id AND t1.pos + 2 = t3.pos
			WHERE 
				t1.token IS NOT NULL AND 
				t2.token IS NOT NULL AND 
				t3.token IS NOT NULL AND
				NOT t1.token ~ '^[0-9]+$' AND 
				NOT t2.token ~ '^[0-9]+$' AND 
				NOT t3.token ~ '^[0-9]+$'
			),
			trigrams AS (
			SELECT 
				lower(token || ' ' || next1 || ' ' || next2) AS trigrama
			FROM ngrams
			)
			SELECT trigrama, COUNT(*) AS total
			FROM trigrams
			GROUP BY trigrama
			ORDER BY total DESC
			LIMIT 31;
			""").fetchdf()


,trigrama,total
0,,349670
1,superior electoral court,17424
2,the superior electoral,17424
3,a decision of,17422
4,in brazil following,17422
5,electoral court tse,17422
6,decision of the,17422
7,was blocked in,17422
8,blocked in brazil,17422
9,of the superior,17422


### 22. As 30 mensagens mais positivas (distintas);

In [116]:
con.execute(r"""
			WITH mensagens_norm AS (
			SELECT
				text_content_anonymous,
				score_sentiment,
				regexp_replace(
				regexp_replace(
					regexp_replace(
					regexp_replace(
						regexp_replace(
						regexp_replace(
							regexp_replace(
							regexp_replace(
								regexp_replace(
								lower(text_content_anonymous),
								'[áàãâä]', 'a', 'g'
								),
								'[éèêë]', 'e', 'g'
							),
							'[íìîï]', 'i', 'g'
							),
							'[óòõôö]', 'o', 'g'
						),
						'[úùûü]', 'u', 'g'
						),
						'[ç]', 'c', 'g'
					),
					'[^a-z0-9 ]', '', 'g'
					),
					'\\s+', ' ', 'g'
				),
				'^\\s+|\\s+$', '', 'g'
				) AS normalized_text
			FROM fakeTelegram
			WHERE text_content_anonymous IS NOT NULL
				AND score_sentiment IS NOT NULL
			),
			ranked AS (
			SELECT 
				text_content_anonymous,
				normalized_text,
				score_sentiment,
				ROW_NUMBER() OVER (PARTITION BY normalized_text ORDER BY score_sentiment DESC) AS rnk
			FROM mensagens_norm
			)
			SELECT text_content_anonymous, score_sentiment
			FROM ranked
			WHERE rnk = 1
			ORDER BY score_sentiment DESC
			LIMIT 30;
			""").fetchdf()


,text_content_anonymous,score_sentiment
0,"Shemoneh Esreh – Amidá – A Grande Oração\n\n ,...",0.9992
1,"MENTIROSA!!!\n\n\nPensa direita, não é correto...",0.9990
2,"Pensa direita, não é correto compartilhar um v...",0.9990
3,"בונה ירושלים, Bo'ne Yetrushalayim - Benção da ...",0.9989
4,Os Sete Samurais \n \nQuando falamos sobre For...,0.9988
5,MANIFESTO À NAÇÃO BRASILEIRA\n\nEM DEFESA DO B...,0.9987
6,BOM DIA A TODOS. ENTENDO QUE ESTAMOS VIVENDO D...,0.9986
7,"Olá ,\n\nUM AMIGO ME PERGUNTOU EM QUEM EU VOTE...",0.9986
8,\nA Viva Esperança\n\nDepois das saudações in...,0.9985
9,Uma forma bem legal\nPara quem ainda não sabe ...,0.9985


### 23. As 30 mensagens mais negativas (distintas);

In [126]:
con.execute(r"""
			WITH mensagens_norm AS (
			SELECT
				text_content_anonymous,
				score_sentiment,
				regexp_replace(
				regexp_replace(
					regexp_replace(
					regexp_replace(
						regexp_replace(
						regexp_replace(
							regexp_replace(
							regexp_replace(
								regexp_replace(
								lower(text_content_anonymous),
								'[áàãâä]', 'a', 'g'
								),
								'[éèêë]', 'e', 'g'
							),
							'[íìîï]', 'i', 'g'
							),
							'[óòõôö]', 'o', 'g'
						),
						'[úùûü]', 'u', 'g'
						),
						'[ç]', 'c', 'g'
					),
					'[^a-z0-9 ]', '', 'g'
					),
					'\\s+', ' ', 'g'
				),
				'^\\s+|\\s+$', '', 'g'
				) AS normalized_text
			FROM fakeTelegram
			WHERE text_content_anonymous IS NOT NULL
				AND score_sentiment IS NOT NULL
			),
			ranked AS (
			SELECT 
				*,
				ROW_NUMBER() OVER (
				PARTITION BY normalized_text 
				ORDER BY score_sentiment DESC, LENGTH(text_content_anonymous) DESC
				) AS rnk
			FROM mensagens_norm
			)
			SELECT text_content_anonymous, score_sentiment
			FROM ranked
			WHERE rnk = 1
			ORDER BY score_sentiment
			LIMIT 30;
			""").fetchdf()


,text_content_anonymous,score_sentiment
0,CAPITÃOZINHO DE MERDA 💩\nCAPITÃOZINHO DE MERDA...,-1.0000
1,Intervenção é meu pau na tua mão\nIntervenção ...,-0.9999
2,ai ta na rua da um joinha 👍Quem ai ta na rua d...,-0.9999
3,"Gente, a Zambelli nos abandonou! Traiu a nossa...",-0.9999
4,ME CÚ NO PAU DO BOLSONARO \nME CÚ NO PAU DO BO...,-0.9999
5,Intervenção é meu pau na tua mão\nIntervenção ...,-0.9999
6,fraude fraude fraude fraude fraude fraude frau...,-0.9999
7,"E nós aqui nos sacrificando, aguentando sol, c...",-0.9999
8,nha 👍Quem ai ta na rua da um joinha 👍Quem ai t...,-0.9998
9,ta na rua da um joinha 👍Quem ai ta na rua da u...,-0.9998


In [125]:
con.execute(r"""
			WITH mensagens_norm AS (
			SELECT
				text_content_anonymous,
				score_sentiment,
				regexp_replace(
				regexp_replace(
					regexp_replace(
					regexp_replace(
						regexp_replace(
						regexp_replace(
							regexp_replace(
							regexp_replace(
								regexp_replace(
								regexp_replace(
									lower(text_content_anonymous),
									'[áàãâä]', 'a', 'g'
								),
								'[éèêë]', 'e', 'g'
								),
								'[íìîï]', 'i', 'g'
							),
							'[óòõôö]', 'o', 'g'
							),
							'[úùûü]', 'u', 'g'
						),
						'[ç]', 'c', 'g'
						),
						'[^a-z0-9 ]', '', 'g'
					),
					'\\s+', ' ', 'g'
					),
					'^\\s+|\\s+$', '', 'g'
				),
				'\\b(\\w+)( \\1\\b)+', '\\1', 'g'
				) AS normalized_text
			FROM fakeTelegram
			WHERE text_content_anonymous IS NOT NULL
				AND score_sentiment IS NOT NULL
			),
			agrupada AS (
			SELECT *,
				CASE
				WHEN normalized_text LIKE '%na rua da um joinha%' THEN 'grupo_joinha'
				WHEN normalized_text LIKE '%intervencao e meu pau%' THEN 'grupo_intervencao'
				WHEN normalized_text LIKE '%fraude%' AND LENGTH(normalized_text) < 100 THEN 'grupo_fraude'
				WHEN normalized_text LIKE '%os covardes e anti patriotas%' THEN 'grupo_covardes'
				ELSE normalized_text
				END AS grupo_final
			FROM mensagens_norm
			),
			ranked AS (
			SELECT 
				*,
				ROW_NUMBER() OVER (
				PARTITION BY grupo_final 
				ORDER BY score_sentiment DESC, LENGTH(text_content_anonymous) DESC
				) AS rnk
			FROM agrupada
			)
			SELECT text_content_anonymous, score_sentiment
			FROM ranked
			WHERE rnk = 1
			ORDER BY score_sentiment
			LIMIT 30;
			""").fetchdf()


,text_content_anonymous,score_sentiment
0,CAPITÃOZINHO DE MERDA 💩\nCAPITÃOZINHO DE MERDA...,-1.0000
1,fraude fraude fraude fraude fraude fraude frau...,-0.9999
2,"E nós aqui nos sacrificando, aguentando sol, c...",-0.9999
3,"Gente, a Zambelli nos abandonou! Traiu a nossa...",-0.9999
4,ME CÚ NO PAU DO BOLSONARO \nME CÚ NO PAU DO BO...,-0.9999
5,Sobre a morte como política global e a necessi...,-0.9998
6,OS COVARDES E ANTI PATRIOTAS BOLSONARISTAS\n\n...,-0.9997
7,"Precisamos nos unir imediatamente, todos os ca...",-0.9996
8,FRAUDE FRAUDE FRAUDE FRAUDE FRAUDE FRAUDE FRAU...,-0.9996
9,EU PAU Ó 🫴🫴🫴🫴🫴🫴🇧🇷🇧🇷🇧🇷🇧🇷🇧🇷🇧🇷🇧🇷ATENÇÃO !!!!!!! 🚨...,-0.9994


### 24. O usuário mais otimista;

In [128]:
con.execute(r"""
			SELECT id_member_anonymous, AVG(score_sentiment) AS media_sentimento, COUNT(*) AS total_mensagens
			FROM fakeTelegram
			WHERE id_member_anonymous IS NOT NULL
			AND score_sentiment IS NOT NULL
			GROUP BY id_member_anonymous
			ORDER BY media_sentimento DESC
			LIMIT 1;
			""").fetchdf()

,id_member_anonymous,media_sentimento,total_mensagens
0,016723c9eb650754d3dfe9b3ba69e5c2,0.9953,1


### 25. O usuário mais pessimista;

In [129]:
con.execute(r"""
			SELECT id_member_anonymous, AVG(score_sentiment) AS media_sentimento, COUNT(*) AS total_mensagens
			FROM fakeTelegram
			WHERE id_member_anonymous IS NOT NULL
			AND score_sentiment IS NOT NULL
			GROUP BY id_member_anonymous
			ORDER BY media_sentimento
			LIMIT 1;
			""").fetchdf()

,id_member_anonymous,media_sentimento,total_mensagens
0,243a43d802953032e1ea9a363546c3b8,-0.9992,1


### 26. As 30 maiores mensagens;

In [130]:
con.execute(r"""
			SELECT text_content_anonymous, LENGTH(text_content_anonymous) AS tamanho
			FROM fakeTelegram
			WHERE text_content_anonymous IS NOT NULL
			ORDER BY tamanho DESC
			LIMIT 30;
			""").fetchdf()


,text_content_anonymous,tamanho
0,"1: Aconteceu também naqueles dias, quando não ...",4096
1,Trecho do livro “Prática da Conspiração A Agen...,4096
2,CCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHOO...,4096
3,"1: Aconteceu também naqueles dias, quando não ...",4096
4,1: Naqueles dias não havia melech em Yisra-EL;...,4096
5,"1: Sansão foi a Gaza, e viu ali uma prostituta...",4096
6,CCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHOO...,4096
7,Trecho do livro “Prática da Conspiração A Agen...,4096
8,COMO O SOCIALISMO DESTRUIU A VENEZUELA EM POUC...,4096
9,[Encaminhado de FYI-Brasil (Sú)]\n🚨ELEIÇÕES DE...,4096


In [134]:
con.execute(r"""
			WITH mensagens_norm AS (
			SELECT
				text_content_anonymous,
				LENGTH(text_content_anonymous) AS tamanho,
				regexp_replace(
				regexp_replace(
					regexp_replace(
					regexp_replace(
						regexp_replace(
						regexp_replace(
							regexp_replace(
							regexp_replace(
								regexp_replace(
								regexp_replace(
									lower(text_content_anonymous),
									'[áàãâä]', 'a', 'g'
								),
								'[éèêë]', 'e', 'g'
								),
								'[íìîï]', 'i', 'g'
							),
							'[óòõôö]', 'o', 'g'
							),
							'[úùûü]', 'u', 'g'
						),
						'[ç]', 'c', 'g'
						),
						'[^a-z0-9 ]', '', 'g'
					),
					'\\s+', ' ', 'g'
					),
					'^\\s+|\\s+$', '', 'g'
				),
				'\\b(\\w+)( \\1\\b)+', '\\1', 'g'
				) AS normalized_text
			FROM fakeTelegram
			WHERE text_content_anonymous IS NOT NULL
			),
			ranked AS (
			SELECT
				text_content_anonymous,
				tamanho,
				normalized_text,
				ROW_NUMBER() OVER (
				PARTITION BY normalized_text 
				ORDER BY tamanho DESC
				) AS rnk
			FROM mensagens_norm
			)
			SELECT text_content_anonymous, tamanho
			FROM ranked
			WHERE rnk = 1
			ORDER BY tamanho DESC
			LIMIT 30;
			""").fetchdf()


,text_content_anonymous,tamanho
0,CCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHOO...,4096
1,1: Naqueles dias não havia melech em Yisra-EL;...,4096
2,COMO O SOCIALISMO DESTRUIU A VENEZUELA EM POUC...,4096
3,ROIA♘ ♞HERON★�CAVALO★§�DE★§�§TROIA♘ ♞HERON★�CA...,4096
4,"1: Aconteceu também naqueles dias, quando não ...",4096
5,"1: Sansão foi a Gaza, e viu ali uma prostituta...",4096
6,PROPOSTAS DE BOLSONARO PARA 2023 - POSTADAS AG...,4096
7,Trecho do livro “Prática da Conspiração A Agen...,4096
8,[Encaminhado de FYI-Brasil (Sú)]\n🚨ELEIÇÕES DE...,4096
9,Sobre a morte como política global e a necessi...,4095


### 27. As 30 menores mensagens;

In [135]:
con.execute(r"""
			WITH mensagens_norm AS (
			SELECT
				text_content_anonymous,
				LENGTH(text_content_anonymous) AS tamanho,
				regexp_replace(
				regexp_replace(
					regexp_replace(
					regexp_replace(
						regexp_replace(
						regexp_replace(
							regexp_replace(
							regexp_replace(
								regexp_replace(
								regexp_replace(
									lower(text_content_anonymous),
									'[áàãâä]', 'a', 'g'
								),
								'[éèêë]', 'e', 'g'
								),
								'[íìîï]', 'i', 'g'
							),
							'[óòõôö]', 'o', 'g'
							),
							'[úùûü]', 'u', 'g'
						),
						'[ç]', 'c', 'g'
						),
						'[^a-z0-9 ]', '', 'g'
					),
					'\\s+', ' ', 'g'
					),
					'^\\s+|\\s+$', '', 'g'
				),
				'\\b(\\w+)( \\1\\b)+', '\\1', 'g'
				) AS normalized_text
			FROM fakeTelegram
			WHERE text_content_anonymous IS NOT NULL
			),
			ranked AS (
			SELECT
				text_content_anonymous,
				tamanho,
				normalized_text,
				ROW_NUMBER() OVER (
				PARTITION BY normalized_text 
				ORDER BY tamanho DESC
				) AS rnk
			FROM mensagens_norm
			)
			SELECT text_content_anonymous, tamanho
			FROM ranked
			WHERE rnk = 1
			ORDER BY tamanho
			LIMIT 30;
			""").fetchdf()


,text_content_anonymous,tamanho
0,Q,1
1,f,1
2,0,1
3,B,1
4,S,1
5,D,1
6,M,1
7,o,1
8,Hi,2
9,BB,2


### 28. O dia em que foi publicado a maior quantidade de mensagens;

In [137]:
con.execute(r"""
			SELECT CAST(date_message AS DATE) AS data, COUNT(*) AS total_mensagens
			FROM fakeTelegram
			WHERE date_message IS NOT NULL
			GROUP BY data
			ORDER BY total_mensagens DESC
			LIMIT 1;
			""").fetchdf()


,data,total_mensagens
0,2022-10-02,25243


### 29. As mensagens que possuem as palavras “FACÇÃO” e “CRIMINOSA”;

In [138]:
con.execute(r"""
			SELECT text_content_anonymous
			FROM fakeTelegram
			WHERE text_content_anonymous ILIKE '%facção%'
			AND text_content_anonymous ILIKE '%criminosa%';
			""").fetchdf()


,text_content_anonymous
0,"Pensem!!!!!!'Se Houve fraude ,Nada foi feito ..."
1,#lulaladrãoseulugarénaprisão \n#PTFACÇÃOCRIMIN...
2,"Se depois de ler esse resumo do Plano do PT, v..."
3,"Se depois de ler esse resumo do Plano do PT, v..."
4,É exatamente isso que a facção criminosa CHAM...
...,...
134,😣🤯🤐 Alexandre de Moraes (Secretário-Geral do P...
135,😣🤯🤐 Alexandre de Moraes (Secretário-Geral do P...
136,😣🤯🤐 Alexandre de Moraes (Secretário-Geral do P...
137,Agressões covardes é termos uma facção crimino...


### 30. As mensagens que possuem a palavra “SEGURANÇA”.

In [139]:
con.execute(r"""
SELECT  text_content_anonymous
FROM fakeTelegram
WHERE text_content_anonymous ILIKE '%segurança%';
""").fetchdf()


,text_content_anonymous
0,🌳 Mato Grosso intensifica ações contra crimes ...
1,⚡️🇧🇷 Actus Liberandi - Ordem De Cristo 🇧🇷\n\n✨...
2,⚡️🇧🇷 Actus Liberandi - Ordem De Cristo 🇧🇷\n\n✨...
3,No Telegram entrem em Configurações > Privacid...
4,No Telegram entrem em Configurações > Privacid...
...,...
2795,Profissão médica instada a dizer às mulheres a...
2796,"TSE, o relatório das Forças Armadas e o caso M..."
2797,"TSE, o relatório das Forças Armadas e o caso M..."
2798,✅ Notícias do Dia - Ano 6 n° 2183\nDia 11 de ...
